# Notebook for looking at photon yield / percentage for diff theta, phi

In [76]:
#imports
import numpy as np
import uproot as up
import pandas as pd
import matplotlib.pyplot as plot
from scipy.stats import norm
import sympy

In [122]:
particle = "mu"
energy = "5"
color_dict = {
    "pi" : "red",
    "mu" : "blue"
}

# uproot_path = f"/cwork/rck32/eic/work_eic/root_files/June_13/variation/full_{particle}_{energy}GeV_10k.edm4hep.root:events"
uproot_path = f"/cwork/rck32/eic/work_eic/root_files/June_14/variation/full_{particle}_{energy}GeV_1000.edm4hep.root:events"
events = up.open(uproot_path)

x_pos_branch = events["HcalBarrelHits/HcalBarrelHits.position.x"].array(library='np')
EDep_branch = events["HcalBarrelHits.EDep"].array(library='np')
Pathlength_branch = events["HcalBarrelHits.pathLength"].array(library='np')
Hits_MC_idx_branch = events["_HcalBarrelHits_MCParticle.index"].array(library='np')
MC_parents = events["_MCParticles_parents.index"].array(library='np')
MC_daughters = events["MCParticles.daughters_end"].array(library='np')
PDG_branch = events["MCParticles.PDG"].array(library='np')
MC_endpoint_x_branch = events["MCParticles.endpoint.x"].array(library='np')
layer_map = [1830.8000, 1841.4000, 1907.5, 1918.1,1984.1999, 1994.8000, 2060.8999,2071.5,2137.6001,2148.1999,2214.3000,2224.8999,2291,2301.6001,2367.6999,2378.3000,2444.3999,2455,2521.1001,2531.6999,2597.8000,2608.3999,2674.5,2685.1001,2751.1999,2761.8000,2827.8999,2838.5]    

In [81]:
def get_num_layers_traversed(x_pos):
    #This map contains the midpoints of each layer - each layer is 1cm in width, so we can check the 0.5cm on each side to see if a hit was in the layer
    layer_map = [1830.8000, 1841.4000, 1907.5, 1918.1,1984.1999, 1994.8000, 2060.8999,2071.5,2137.6001,2148.1999,2214.3000,2224.8999,2291,2301.6001,2367.6999,2378.3000,2444.3999,2455,2521.1001,2531.6999,2597.8000,2608.3999,2674.5,2685.1001,2751.1999,2761.8000,2827.8999,2838.5]
    for layer_idx in range(len(layer_map)):
        #check if particle hit within layer
        if ((x_pos >= layer_map[layer_idx] - 5) and (x_pos <= layer_map[layer_idx] + 5)):
            return layer_idx
    #if no layers hit, send error code
    return -1
#theta = 0: px, py = 0
#theta - xz plane
#phi - xy plane
#https://stackoverflow.com/questions/4116658/faster-numpy-cartesian-to-spherical-coordinate-conversion
def theta_func(px,py,pz):
    return sympy.acos(pz / (px ** 2 + py ** 2 + pz ** 2)) * 180 / np.pi
def phi_func(px,py,pz):
    return sympy.atan2(py,px) * 180 / np.pi


def Efunc(px,py,pz,m):
    return np.sqrt(px**2 + py**2 + pz**2 + m**2)
    
class PVect:
    def __init__(self):
        self.px = 0
        self.py = 0
        self.pz = 0
        self.theta = 0
        self.phi = 0
        self.E = 0
        self.M = 0
    def setVector(self,px,py,pz,m):
        self.px = px
        self.py = py
        self.pz = pz
        self.M = m
        self.E = Efunc(px,py,pz,m)
        self.theta = theta_func(px,py,pz)
        self.phi = phi_func(px,py,pz)

In [82]:
MC_px = events["MCParticles.momentum.x"].array(library='np')
MC_py = events["MCParticles.momentum.y"].array(library='np')
MC_pz = events["MCParticles.momentum.z"].array(library='np')
MC_m = events["MCParticles.mass"].array(library='np')

In [123]:
theta_min = 67
theta_max = 113
phi_min = -22
phi_max = 22

num_MC = 0
num_hits = 0
energy_dep = 0
dep_count = 0
hits_per_photon = []
num_events = len(EDep_branch)
for event_idx in range(len(EDep_branch)):
    if(event_idx > 100):
        break
    num_MC = 0
    primary = PVect()
    primary.setVector(MC_px[event_idx][0],MC_py[event_idx][0],MC_pz[event_idx][0],MC_m[event_idx][0])
    print(f"event # {event_idx}: theta = {primary.theta}; phi = {primary.phi}")

event # 0: theta = 89.3530148321767; phi = 4.01573352795426
event # 1: theta = 89.0114218371604; phi = 8.46837938248004
event # 2: theta = 87.6057750995283; phi = -2.01202528670779
event # 3: theta = 87.8552717378511; phi = 7.33920036633500
event # 4: theta = 90.5827053247652; phi = -7.02444096426377
event # 5: theta = 87.1267667429213; phi = 21.8044846258304
event # 6: theta = 92.8348600306281; phi = -11.1865578922539
event # 7: theta = 93.7274121072484; phi = 21.5408988224047
event # 8: theta = 90.3748902186229; phi = 1.20209508008446
event # 9: theta = 93.4148532986721; phi = 21.1133981929454
event # 10: theta = 89.2519254602501; phi = 19.7671867285016
event # 11: theta = 91.8637791020725; phi = 1.06216575346831
event # 12: theta = 88.9600627219755; phi = 10.5618426842637
event # 13: theta = 88.2396864925557; phi = 6.60582140210935
event # 14: theta = 90.2870852115768; phi = 21.0286917046788
event # 15: theta = 92.6879967932876; phi = -10.6361454283866
event # 16: theta = 88.7347431

In [8]:
num_MC = 0
num_hits = 0
energy_dep = 0
dep_count = 0
hits_per_photon = []
num_events = len(EDep_branch)
for event_idx in range(len(EDep_branch)):
  
    num_MC = 0
    for i in range(len(PDG_branch[event_idx])):
        if(PDG_branch[event_idx][i] == -22 and (i < MC_daughters[event_idx][0])):num_MC+= 1
    if(num_MC == 0):
        print(f"skipping event #{event_idx}, no optph found")
    else:
        hits_per_photon.append(len(EDep_branch[event_idx]) / num_MC)
#     if(not (event_idx % 100)):
#         print(f"{event_idx // 100} % done")
print(f"% of photons hitting = {sum(hits_per_photon) / len(hits_per_photon) * 100}%")

% of photons hitting = 0.8661019396447213%


# Calculate % of photons hitting sensor for diff theta phi bins

In [110]:
n_bins = 10

theta_min = 68
theta_max = 112
phi_min = -22
phi_max = 22

theta_range = theta_max - theta_min
phi_range = phi_max - phi_min

phi_bins = np.linspace(phi_min,phi_max,n_bins+1)
theta_bins = np.linspace(theta_min,theta_max,n_bins+1)

In [111]:
#works only for evenly spaced bins
def findBin(val,bins):
    diff = bins[1] - bins[0]
    rel_dist = val - bins[0]
    mod = np.floor(rel_dist / diff)
    return int(mod)

In [112]:
theta_percent = [[] for i in range(n_bins)]
phi_percent = [[] for i in range(n_bins)]

In [113]:
theta_MC = np.zeros(n_bins)
phi_MC = np.zeros(n_bins)

theta_hits = np.zeros(n_bins)
phi_hits = np.zeros(n_bins)

theta_counts = np.zeros(n_bins)
phi_counts = np.zeros(n_bins)

In [114]:
num_MC = 0
num_hits = 0
energy_dep = 0
dep_count = 0
hits_per_photon = []
num_events = len(EDep_branch)
break_val = 1000
for event_idx in range(len(EDep_branch)):
    if(event_idx % (break_val // 100))
    if(event_idx > 1000):
        break
    num_MC = 0
    primary = PVect()
    primary.setVector(MC_px[event_idx][0],MC_py[event_idx][0],MC_pz[event_idx][0],MC_m[event_idx][0])
    phi_bin = findBin(primary.phi,phi_bins)
    theta_bin = findBin(primary.theta,theta_bins)
    phi_counts[phi_bin]+=1
    theta_counts[theta_bin]+=1
#     print(f"theta, phi: {primary.theta}, {primary.phi} | px,pz = {primary.px,primary.pz} | theta, phi bins: {theta_bin},{phi_bin}")
    for i in range(len(PDG_branch[event_idx])):
        if(PDG_branch[event_idx][i] == -22 and (i < MC_daughters[event_idx][0])):
            theta_MC[theta_bin] += 1
            phi_MC[phi_bin] += 1
    for hit in range(len(EDep_branch[event_idx])):
        #check if hit is from optph
        if(PDG_branch[event_idx][Hits_MC_idx_branch[event_idx][hit]] == -22):
            theta_hits[theta_bin] += 1
            phi_hits[phi_bin] += 1
for i in range(n_bins):
    #if no MCParticles, then just set to -1
    if(not len(theta_MC)):
        theta_percent[i] = -1
    else:
        theta_percent[i].append(theta_hits[i] / theta_MC[i])
    if(not len(phi_MC)):
        phi_percent[i] = -1
    else:
        phi_percent[i].append(phi_hits[i] / phi_MC[i])

<ipython-input-114-84bb065d2f7f>:32: RuntimeWarning: invalid value encountered in double_scalars
  theta_percent[i].append(theta_hits[i] / theta_MC[i])


In [115]:
print("theta:           phi: ")
for i in range(n_bins):
    print(f"bin #{i}: {theta_counts[i]}    |   {phi_counts[i]}")

theta:           phi: 
bin #0: 0.0    |   99.0
bin #1: 0.0    |   104.0
bin #2: 0.0    |   106.0
bin #3: 0.0    |   89.0
bin #4: 496.0    |   92.0
bin #5: 505.0    |   104.0
bin #6: 0.0    |   111.0
bin #7: 0.0    |   94.0
bin #8: 0.0    |   95.0
bin #9: 0.0    |   107.0


In [120]:
for i in range(n_bins):
    print(f"bin #{i}: {phi_percent[i][0] * 100}%")

bin #0: 1.217720287219985%
bin #1: 1.3867305159333871%
bin #2: 1.1959096158699971%
bin #3: 0.7840685903277309%
bin #4: 0.8397362960786275%
bin #5: 1.1291082481763683%
bin #6: 0.9524290644134168%
bin #7: 0.9171510648072877%
bin #8: 1.0116415446288523%
bin #9: 1.059408560544408%


# OLD EXTRA

In [117]:
momenta = [[ 0.888, -0.296, -0.352],
[0.963,  0.268, -0.025],
[0.992,  0.066, -0.106],
 [0.966, -0.241,  0.094]]

In [121]:
for i in range(len(momenta)):
    print(f"theta: {theta_func(momenta[i][0],momenta[i][1],momenta[i][2])}")

theta: 110.608314015655
theta: 91.4328045623539
theta: 96.0868856845777
theta: 84.6066284120157


In [127]:
print(f"theta: {theta_func(0.981,-0.086,0.174)} | phi: {phi_func(0.981,-0.086,0.174)}")

theta: 79.9798645288412 | phi: -5.01006324057405


In [128]:
print(f"theta: {theta_func(4.9053010,-0.429158,0.8682408)} | phi: {phi_func(4.9053010,-0.429158,0.8682408)}")

theta: 88.0097381614281 | phi: -4.99999734919290


In [129]:
4.9053010 ** 2 + (-0.429158) ** 2 + (0.8682408) ** 2

24.999996576349638